In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📚 Solow Growth Model with Technological Progress

The Solow growth model is a fundamental framework for understanding long-run economic growth. It explains how capital accumulation, labor force growth, and technological progress interact to determine the level and growth rate of output per worker.

This simulation focuses on the core Solow model incorporating **labor-augmenting technological progress** ($A$). The production function is typically assumed to be Cobb-Douglas with constant returns to scale:

$$Y = K^{\alpha} (AL)^{1 - \alpha}$$

- $Y$: Total output
- $K$: Physical capital stock
- $L$: Labor force
- $A$: Level of technology (labor-augmenting or TFP, depending on interpretation)
- $\alpha$: Capital's share of income ($0 < \alpha < 1$)

The term $AL$ is often called **effective labor**, representing the labor force adjusted for its productivity level due to technology $A$.
*(Note: While the original notebook mentioned human capital $H$, this simulation uses the standard $AL$ framework. $H$ can be thought of as implicitly included within $A$ or as augmenting $L$ similarly to $A$. A separate dynamic model is needed to track $H$ accumulation explicitly).*

# ⚙️ Dynamics in Intensive Form (Per Effective Worker)

To analyze the model's dynamics and convergence to a steady state, we express it in terms of quantities **per effective worker**. Let:
- $k = K / (AL)$: Capital per effective worker
- $y = Y / (AL)$: Output per effective worker

Substituting into the production function gives:
$$y = \frac{K^{\alpha} (AL)^{1 - \alpha}}{AL} = \left(\frac{K}{AL}\right)^{\alpha} = k^{\alpha}$$

The fundamental dynamic equation describing how capital per effective worker ($k$) evolves over time is:

$$\dot{k} = s k^{\alpha} - (\delta + n + g) k$$
or in discrete time (as used in the simulation):
$$k_{t+1} - k_t \approx s k_t^{\alpha} - (\delta + n + g) k_t$$

Where:
- $s$: Savings rate (fraction of output invested)
- $\delta$: Depreciation rate of capital
- $n$: Growth rate of the labor force ($L$)
- $g$: Growth rate of technology ($A$)

The term $s k^{\alpha}$ represents investment per effective worker (saving equals investment), and $(\delta + n + g) k$ represents the "breakeven investment" needed to keep $k$ constant despite depreciation ($\delta$), growth in effective labor due to population growth ($n$), and growth in effective labor due to technological progress ($g$).

# ⚖️ Steady State & Simulation

The economy converges to a **steady state** where capital per effective worker ($k$) is constant ($\dot{k} = 0$). This occurs when actual investment equals breakeven investment:

$$s (k^*)^{\alpha} = (\delta + n + g) k^*$$

Solving for the steady-state capital per effective worker ($k^*$):

$$k^* = \left( \frac{s}{\delta + n + g} \right)^{\frac{1}{1-\alpha}}$$

The corresponding steady-state output per effective worker ($y^*$) is:

$$y^* = (k^*)^{\alpha} = \left( \frac{s}{\delta + n + g} \right)^{\frac{\alpha}{1-\alpha}}$$

**Key Implications:**

* In the steady state, $k$ and $y$ are constant.
* Output per *worker* ($Y/L = y \times A$) grows at the rate $g$ (rate of technological progress).
* Total output ($Y = y \times A \times L$) grows at the rate $n+g$.

The simulation below plots the time path of capital per effective worker ($k_t$) and output per effective worker ($y_t$) as they converge towards their steady-state values ($k^*, y^*$). Adjust the parameters to see how they affect the steady-state levels and the convergence speed.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider # Use IntSlider for T
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def solow_model_simulation(s=0.2, delta=0.05, alpha=0.33, n=0.01, g=0.02, k0=1.0, T=100):
    """
    Simulates the Solow Growth Model in intensive form (per effective worker).
    Y = K^alpha * (AL)^(1-alpha) => y = k^alpha
    k_dot = s*k^alpha - (delta + n + g) * k

    Args:
        s (float): Savings rate.
        delta (float): Depreciation rate.
        alpha (float): Capital share (output elasticity of capital).
        n (float): Population / Labor force growth rate.
        g (float): Technological progress growth rate (labor-augmenting).
        k0 (float): Initial capital per effective worker (K/AL).
        T (int): Time Horizon (number of periods).
    """
    # Ensure T is an integer
    T = int(T)
    if T < 5: T = 5

    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)
    s = np.clip(s, 0.01, 0.99)
    delta = max(delta, 0)
    n = max(n, 0) # Assuming non-negative growth rates usually
    g = max(g, 0)
    k0 = max(k0, 0.01) # Start k slightly above zero

    # Calculate steady state
    break_even_rate = delta + n + g
    if break_even_rate <= 0: # Avoid division by zero or invalid exponent
        k_star = np.inf # Or handle as error / very large number
        y_star = np.inf
    else:
        # k_star = (s / break_even_rate)**(1 / (1 - alpha))
        # Calculate safely to avoid potential overflow with large exponents
        exponent = 1.0 / (1.0 - alpha)
        base = s / break_even_rate
        if base <= 0:
             k_star = 0.0 # If s=0 or break_even_rate is huge
        else:
             # Use log-exp trick for potentially large exponents
             log_k_star = exponent * np.log(base)
             # Set a reasonable upper bound to prevent extreme values in plots
             log_k_star = min(log_k_star, 20) # Corresponds to k_star ~ 5e8
             k_star = np.exp(log_k_star)

        y_star = k_star**alpha if k_star > 0 else 0.0


    # --- Simulation ---
    k = np.zeros(T)
    k[0] = k0
    for t in range(T - 1):
        investment = s * k[t]**alpha
        break_even_investment = break_even_rate * k[t]
        # Change in k = investment - break_even_investment
        delta_k = investment - break_even_investment
        k[t+1] = k[t] + delta_k
        # Ensure k doesn't go below a small positive value
        k[t+1] = max(k[t+1], 1e-9)

    # Calculate output per effective worker
    y = k**alpha

    # --- Plotting ---
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    time = np.arange(T)

    # Plot 1: Capital per Effective Worker (k)
    axes[0].plot(time, k, label='k(t)', color='navy', lw=2)
    if np.isfinite(k_star):
        axes[0].axhline(k_star, color='red', linestyle='--', label=f'Steady State k* ≈ {k_star:.2f}')
    axes[0].set_title("Capital per Effective Worker (k = K/AL)")
    axes[0].set_xlabel("Time (t)")
    axes[0].set_ylabel("k(t)")
    axes[0].legend()
    axes[0].grid(True, linestyle='--', alpha=0.7)
    # Dynamic Y limits for k plot
    k_min = 0
    k_max = max(k.max(), k_star if np.isfinite(k_star) else k.max())
    axes[0].set_ylim(k_min, k_max * 1.1)
    axes[0].set_xlim(0, T-1)


    # Plot 2: Output per Effective Worker (y)
    axes[1].plot(time, y, label='y(t)', color='forestgreen', lw=2)
    if np.isfinite(y_star):
        axes[1].axhline(y_star, color='red', linestyle='--', label=f'Steady State y* ≈ {y_star:.2f}')
    axes[1].set_title("Output per Effective Worker (y = Y/AL)")
    axes[1].set_xlabel("Time (t)")
    axes[1].set_ylabel("y(t)")
    axes[1].legend()
    axes[1].grid(True, linestyle='--', alpha=0.7)
    # Dynamic Y limits for y plot
    y_min = 0
    y_max = max(y.max(), y_star if np.isfinite(y_star) else y.max())
    axes[1].set_ylim(y_min, y_max * 1.1)
    axes[1].set_xlim(0, T-1)

    fig.suptitle(f"Solow Model Simulation (s={s:.2f}, δ={delta:.2f}, n={n:.2f}, g={g:.2f}, α={alpha:.2f})", fontsize=14, y=1.0)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

    # --- Display Steady State Info ---
    steady_state_md = f"""
    ### ⚖️ Steady State Values:

    * **Capital per Effective Worker (k\*):** {k_star:.3f}
    * **Output per Effective Worker (y\*):** {y_star:.3f}
    * *(Calculated using: $k^* = (s / (\\delta + n + g))^{{1/(1-\\alpha)}}$ and $y^* = (k^*)^{{\\alpha}}$)*
    * *(Note: The simulation converges towards these values over time)*
    """
    display(Markdown(steady_state_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
interact(
    solow_model_simulation,
    s=FloatSlider(value=0.25, min=0.05, max=0.6, step=0.01, description='Savings Rate (s):', style=style, readout_format='.2f'),
    delta=FloatSlider(value=0.05, min=0.01, max=0.15, step=0.005, description='Depreciation (delta δ):', style=style, readout_format='.3f'),
    alpha=FloatSlider(value=0.33, min=0.1, max=0.8, step=0.01, description='Capital Share (alpha α):', style=style, readout_format='.2f'),
    n=FloatSlider(value=0.01, min=0.0, max=0.05, step=0.002, description='Pop. Growth (n):', style=style, readout_format='.2%'),
    g=FloatSlider(value=0.02, min=0.0, max=0.05, step=0.002, description='Tech Growth (g):', style=style, readout_format='.2%'),
    k0=FloatSlider(value=1.0, min=0.1, max=20.0, step=0.1, description='Initial k (k0):', style=style, readout_format='.1f'),
    T=IntSlider(value=100, min=20, max=500, step=10, description='Time Periods (T):', style=style)
);


<>:128: SyntaxWarning: invalid escape sequence '\*'
<>:126: SyntaxWarning: invalid escape sequence '\*'
<>:128: SyntaxWarning: invalid escape sequence '\*'
<>:126: SyntaxWarning: invalid escape sequence '\*'
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_86667/3654325098.py:128: SyntaxWarning: invalid escape sequence '\*'
  
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_86667/3654325098.py:126: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(steady_state_md))


interactive(children=(FloatSlider(value=0.25, description='Savings Rate (s):', max=0.6, min=0.05, step=0.01, s…

# 📚 Solow Model with Human Capital

Incorporating **human capital** into the Solow model adds a realistic dimension to labor quality.

# 🔍 Equation:
\[
Y = K^{\alpha} (A \cdot H \cdot L)^{1 - \alpha}
\]

Where:
- \( H \): human capital (e.g. schooling, skills)
- \( A \): productivity
- \( L \): labor force
- \( K \): capital

> Human capital raises **effective labor** and helps explain **cross-country income differences**.

# 📈 Growth Implication:
Output grows due to improvements in:
- Physical capital (K)
- Population (L)
- Technology (A)
- Human capital (H)

Even when \( A \) is the same across countries, **differences in H** can explain gaps in output per worker.